# Topic Model of the State Archives of Assyria Letters

This is a work-in-progress Notebook. The number of topics is set with the ntopics variable. There are almost 2500 letters, most of them brief or very brief. The topic model produced from the entire corpus (with 15 or 25 topics) is hard to interpret. One reason may be that the texts are too short. A brief or broken letter with just a few words that happen to be important in a certain topic may come out as one of the most important documents in that topic (because a high percentage of that letter resides in the topic). One solution may be to group (aggregate) letters by metadata, in the file SAAO.csv. One could group the letters by Chapter number (this referes to chapters in the original book publications - assuming that those chapters group texts that are related in a meaningful way)


In [3]:
import pandas as pd
import glob
import re
import numpy as np
import pyLDAvis
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils.fixes import sp_version

# Dependencies and Versions
The following code checks for the versions of the packages pyLDAvis, scikit-learn, and pandas and will install updates when necessary. pyLDAvis 2.0 is incompatible with Pandas 0.19. The MDS and tSNE computations necessary for plotting the distribution of documents in the topic model need scikit-learn 0.18. This notebook was written for Python 3.5.

In [4]:
if sp_version < (0, 18):
    print('scikit-learn version ' + str(sp_version) + '. You need to update to scikit-learn 0.18 or later.')
    y_n = input('do you want to proceed? y/n ')
    if y_n == 'y':
        !pip install -U scikit-learn

pdv = pd.__version__.split('.')
if int(pdv[1]) < 19:
    print('pandas version ' + pd.__version__ + '. You need to update to pandas 0.19 or later.')
    y_n = input('do you want to proceed? y/n ')
    if y_n == 'y':
        !pip install -U pandas

pyLDAvisv = pyLDAvis.__version__.split('.')
if int(pyLDAvisv[0]) < 2 or int(pyLDAvisv[1]) <  1:
    print('pyLDAvis version ' + pyLDAvis.__version__ + '. You need to update to pyLDAvis 2.1.0 or later.')
    y_n = input('do you want to proceed? y/n ')
    if y_n == 'y':
        !pip install -U pyLDAvis


# Read in Data
First read the directory with the State Archives of Assyria letters. These files contain lemmatization in ORACC. The texts list lemmatizations per line.

In [5]:
path =r'../Scrape-Oracc/Output/' # use your path
allFiles = glob.glob(path + "/saao" + "*.txt")
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
saao_data = pd.concat(list_)
saao_data.head()

,id_text,l_no,text,text_name,version
0,saao/saa01/P224485,o 1,awātu[word]N šarru[king]N ana[to]PRP Aššur-šar...,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,NaN
1,saao/saa01/P224485,o 2,šulmu[completeness]N ana[to]PRP libbu[interior...,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,NaN
2,saao/saa01/P224485,o 3,ša[that]REL šapāru[send]V mā[saying]PRP māru[s...,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,NaN
3,saao/saa01/P224485,o 4,Muskaya[Phrygian]EN ina[in]PRP muhhu[skull]N a...,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,NaN
4,saao/saa01/P224485,o 5,Quwaya[from-Quwe]EN ša[that]REL Urik[1]PN ana[...,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,NaN


# Collapse lines to one row per document
In order to transform this DataFrame into a proper input for topic modeling we need to discard the column `l_no` and concatenate all the text that belongs to one letter in a single row. Some lines have no content in the `text` column - these lines need to be dropped.

First select the relevant columns and drop the rows that have no text content.

In [6]:
saao_data = saao_data[['id_text', 'text_name', 'text']]
saao_data = saao_data.dropna()
saao_data.head()

,id_text,text_name,text
0,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,awātu[word]N šarru[king]N ana[to]PRP Aššur-šar...
1,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,šulmu[completeness]N ana[to]PRP libbu[interior...
2,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,ša[that]REL šapāru[send]V mā[saying]PRP māru[s...
3,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,Muskaya[Phrygian]EN ina[in]PRP muhhu[skull]N a...
4,saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,Quwaya[from-Quwe]EN ša[that]REL Urik[1]PN ana[...


Group the rows by `id_text` and apply the `join` function to the `text` column. Transform the aggregated data into a new DataFrame.

In [7]:
saao_byletter = saao_data['text'].groupby(saao_data['id_text']).apply(' '.join)
saao_byletter_df = pd.DataFrame(saao_byletter)
saao_byletter_df.head()

,text
id_text,
saao/saa01/P224485,awātu[word]N šarru[king]N ana[to]PRP Aššur-šar...
saao/saa01/P313416,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313417,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313425,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313427,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...


Create a DataFrame of `id_text` and `text_name` equivalencies, with `id_text` set as index (row names). Then merge this DataFrame with the the `saao_bytext_df` using the indexes.

In [8]:
saao_id_names = saao_data[['id_text', 'text_name']].drop_duplicates().set_index('id_text')
saao_data_df = pd.merge(saao_id_names, saao_byletter_df, right_index=True, left_index=True)
saao_data_df.head()

,text_name,text
id_text,,
saao/saa01/P224485,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...,awātu[word]N šarru[king]N ana[to]PRP Aššur-šar...
saao/saa01/P313416,SAA 01 158. Gold and Silver Objects Sent to th...,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313417,SAA 01 233. More Land to Bel-duri (CT 53 002),ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313425,SAA 01 179. No Iron to the Arabs! (CT 53 010),ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...
saao/saa01/P313427,SAA 01 152. The Affair of Gidgidanu and His Br...,ana[to]PRP šarru[king]N bēlu[lord]N ardu[slave...


In [11]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    import gensim
    from gensim import corpora, models, utils

# Test corpus: first 100 texts
[For test purposes only the first 100 documents are admitted.]

In [12]:
#saao_data_df = saao_data_df[:100]
documents = saao_data_df['text']

# Tokenize and POS-filter
Tokenization is done by splitting on white space. The variable `posfilter` holds the last two characters of lemmatized words with allowed Part of Speech tags. If, for instance, you wish to select Verbs, Adjectives, and Nouns (in Akkadian), posfilter will be `[']n', 'aj', ']v']`. Note that one-character pos-tags need the right bracket!
The POS labels are:
* "n", #Nouns
* "v", #Verbs
* "aj", #Adjectives
* "av", #Adverbs
* "an", #Agricultural Name
* "cn", #Celestial Name
* "dn", #Divine Name
* "en", #Ethnicity Name
* "fn", #Field Name
* "gn", #Geographical Name (lands, etc.)
* "ln", #Lineage Name (ancestral clan)
* "mn", #Month Name
* "on", #Object Name
* "pn", #Personal Name
* "qn", #Quarter (of a city) Name
* "rn", #Royal Name
* "sn", #Settlement Name
* "tn", #Temple Name
* "wn", #Watercourse Name
* "yn", #Year Name
* "nu", #Numeral


In [13]:
posfilter = [']n', ']v', 'en']
#texts = [[word for word in document.lower().split()] for document in documents]
texts = [[word for word in document.lower().split() if word[-2:] in posfilter] for document in documents]

# Stop words

Stop words are very frequent words that are not able to distinguish between topics. This includes, for instance, prepositions - but those have already been filtered out by the POS filter. The following nouns and verbs are too frequent to contribute to the analysis.

In [14]:
stoplist = [
'šarru[king]n',
'bēlu[lord]n',
'libbu[interior]n',
'muhhu[skull]n',
'ardu[slave]n',
'šulmu[completeness]n',
'šapāru[send]v',
'alāku[go]v',
'qabû[say]v',
'pānu[front]n',
'māru[son]n',
'bītu[house]n',
'epēšu[do]v',
'wabālu[bring]v',
'šakānu[put]v',
'amāru[see]v',
'bašû[exist]v',
'našû[lift]v',
'izuzzu[stand]v',
'ūmu[day]n',
'ṭābu[good]aj',
'mādu[many]aj',
'nadānu[give]v',
'tadānu[give]v',
'ṣehru[small]aj',
'mimmû[all]n',
'gimru[totality]n',
'gabbu[totality]n',
'šâlu[ask]v',
'šemû[hear]v',
'ūmu[day]n',
'awātu[word]n',
'erēbu[enter]v']
texts = [[word for word in text if word not in stoplist] for text in texts]

# Dictionary
create the gensim Dictionary and filter for words that are too common or too rare (no_above may be set too low here).

In [15]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)

In [16]:
corpus = [dictionary.doc2bow(doc) for doc in texts]

In [17]:
seed = 42
np.random.seed(seed)
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
ntopics = 10
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus, num_topics=ntopics, id2word = dictionary, passes=50)

List the top words and their probabilities in all topics. Note: the topic numbers here are not the ones used below in the visualizations! (The topics are the same, but not their numbers).

In [18]:
ldamodel.show_topics(ntopics, formatted = False)

[(0,
  [('mātu[land]n', 0.063400252146628852),
   ('ilu[god]n', 0.035541554004914641),
   ('karābu[pray]v', 0.026565829676768227),
   ('abu[father]n', 0.022407490801860695),
   ('qātu[hand]n', 0.018374501989379944),
   ('ahu[brother]n', 0.018278741756977118),
   ('balāṭu[live]v', 0.017324600746473458),
   ('ṭēmu[(fore)thought]n', 0.017047325328619318),
   ('ṣabātu[seize]v', 0.016420937921817043),
   ('wadû[know]v', 0.014811336971763487)]),
 (1,
  [('karābu[pray]v', 0.08417935618216027),
   ('ṭūbu[goodness]n', 0.051566272398177938),
   ('šīru[flesh]n', 0.031197874927567187),
   ('ilu[god]n', 0.024495509577952799),
   ('šattu[year]n', 0.022660625267432441),
   ('dullu[trouble]n', 0.01793755430981802),
   ('aṣû[go-out]v', 0.015335495934939497),
   ('ṭēmu[(fore)thought]n', 0.01332754718264197),
   ('lēʾu[board]n', 0.013201862563547218),
   ('šiāru[morning]n', 0.011529525314980889)]),
 (2,
  [('manû[unit]n', 0.049834705634218553),
   ('eqlu[field]n', 0.037546784668346461),
   ('mahāru[face]

# pyLDAvis
Use pyLDAvis to visualize the topic model. By default, pyLDAvis will order the topics by [prevalence](https://github.com/bmabey/pyLDAvis/issues/59) (topic 1 is the most prevalent topic). That means that the topic numbers in the visualization do not agree with the topic numbers in the lda model. To prevent this behaviour one may use `sort_topics=False` in the `prepare` command. The advantage of ordering the topics by prevalence, however, is that new instances of the lda model are more comparable (that is, the same topic will receive the same number). Note that the library was written in Java for R, and so the numbering in the visualization begins with 1 (not with 0). The topic numbers in the Document/Topic and Topic/Term matrices below will be adjusted to be compatible with the pyLDAvis visualization.

In [19]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(vis, 'saao.html')
pyLDAvis.display(vis)

# Create Document/Topic Table
Create a DataFrame that gives the probability of each topic for each document.

In [20]:
list_of_doctopics = [ldamodel.get_document_topics(corpus[i], minimum_probability=0) for i in range(len(corpus))]
list_of_probabilities = [[probability for label,probability in distribution] for distribution in list_of_doctopics]
df_list = [pd.DataFrame(list_of_probabilities[i]).transpose() for i in range(len(corpus))]
doc_topic_df = pd.concat(df_list)
doc_topic_df = doc_topic_df.set_index(saao_data_df.index)
doc_topic_df.head()

,0,1,2,3,4,5,6,7,8,9
id_text,,,,,,,,,,
saao/saa01/P224485,0.001075,0.001075,0.001075,0.001075,0.990321,0.001076,0.001075,0.001075,0.001075,0.001075
saao/saa01/P313416,0.002941,0.002943,0.002942,0.002941,0.002941,0.002941,0.002942,0.973525,0.002941,0.002941
saao/saa01/P313417,0.002174,0.002174,0.241682,0.420342,0.112929,0.212001,0.002174,0.002174,0.002174,0.002175
saao/saa01/P313425,0.001352,0.117684,0.151766,0.179846,0.001352,0.366996,0.001352,0.117920,0.001351,0.060380
saao/saa01/P313427,0.301108,0.004168,0.004168,0.322430,0.004168,0.004168,0.347289,0.004167,0.004167,0.004168


Rename the topics `topic 1` to `topic n` in accordance with the pyLDAvis visualization.

In [21]:
topics = ['topic ' + str(i+1) for i in range(ntopics)]

In [22]:
doc_topic_df.columns = topics
doc_topic_df['text_name'] = saao_data_df['text_name']
doc_topic_df.head()

,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,topic 10,text_name
id_text,,,,,,,,,,,
saao/saa01/P224485,0.001075,0.001075,0.001075,0.001075,0.990321,0.001076,0.001075,0.001075,0.001075,0.001075,SAA 01 001. Midas Of Phrygia Seeks Detente (NL...
saao/saa01/P313416,0.002941,0.002943,0.002942,0.002941,0.002941,0.002941,0.002942,0.973525,0.002941,0.002941,SAA 01 158. Gold and Silver Objects Sent to th...
saao/saa01/P313417,0.002174,0.002174,0.241682,0.420342,0.112929,0.212001,0.002174,0.002174,0.002174,0.002175,SAA 01 233. More Land to Bel-duri (CT 53 002)
saao/saa01/P313425,0.001352,0.117684,0.151766,0.179846,0.001352,0.366996,0.001352,0.117920,0.001351,0.060380,SAA 01 179. No Iron to the Arabs! (CT 53 010)
saao/saa01/P313427,0.301108,0.004168,0.004168,0.322430,0.004168,0.004168,0.347289,0.004167,0.004167,0.004168,SAA 01 152. The Affair of Gidgidanu and His Br...


# Create Topic / Term table
This is a table with N rows (the number of topics) and M columns (the number of individual terms in the Dictionary). The table indicates the probability of each term in each topic.

In [23]:
topic_term = ldamodel.show_topics(ntopics, formatted=False, num_words=len(dictionary))

The object `topic_term` is a list of list. Each topic is represented by a list of tuples in the form `(word, probability)`. The following code pulls out the probabilities for each word in each topic (`topic_term[i][1]`) and creates a list of DataFrames with the words as index (rows) and the probabilities as the only column. Each Dataframe is transposed and the DataFrames are concatenated to a single DataFrame. 

In [24]:
topic_term_list = [pd.DataFrame(topic_term[i][1]).set_index(0).transpose() for i in range(0, ntopics)]
topic_term_df = pd.concat(topic_term_list, ignore_index=True)
topic_term_df['topics'] = topics
topic_term_df = topic_term_df.set_index('topics')

The topics (rows) are numbered 0-14 in the (random) order of the lda model. The array `prevalences` is used to re-arrange the topics in the order of prevalence (as is done above for the document/topic matrix). The re-arranged topics (rows) are now re-labeled `topic 1` to `topic 15` in accordance with their numbering in the pyLDAvis visualization.

In [25]:
#topic_term_df['ēkallu[palace]n']

# Visualize the Documents 1.: Using MDS

While pyLDAvis is an excellent tool for exploring the topic/term aspect of a topic model (the words and their probabilities in each topic) it does not provide access to the document/topic aspect (the probability distribution of topics in each document). The visualization below plots all the documents according to their distances (using Multi-Dimensional Scaling) in the Document/Topic DataFrame. Each document (dot) is colored according to the most prevalent topic and the size of the dot represents the probability of the most prevalent topic in that document.

In [26]:
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import MDS

Compute the distances between each of the documents. Use either the Document/Topic Dataframe or the Document/Term Dataframe (constructed below) to measure distance.

In [27]:
cv = CountVectorizer(analyzer='word', token_pattern=r'[^ ]+', vocabulary=list(topic_term_df.columns.values))
saao_dtm = cv.fit_transform(saao_data_df['text'])
saao_dtm_df = pd.DataFrame(saao_dtm.toarray(), columns = cv.get_feature_names(), index = saao_data_df.index.values)
saao_dtm_df.head()

,abati[(meaning-unknown)]n,abbūtu[fatherhood]n,abdudaya[abdudaean]en,abiktu[defeat]n,abku[captive]n,abnu[stone]n,abu[father]n,abullu[(city)-gate]n,abāku[lead-away]v,abālu[dry-(up)]v,...,ṭābtu[goodness]n,ṭābtu[salt]n,ṭābtānu[doer-of-good]n,ṭābtūtu[goodwill]n,ṭātu[bribe]n,ṭēmu[(fore)thought]n,ṭēmūtu[of-order]n,ṭīdu[clay]n,ṭīmu[yarn]n,ṭūbu[goodness]n
saao/saa01/P224485,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
saao/saa01/P313416,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
saao/saa01/P313417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
saao/saa01/P313425,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
saao/saa01/P313427,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#dist = squareform(pdist(saao_dtm_df))
dist = squareform(pdist(doc_topic_df.drop('text_name', axis=1)))

Compute the position of each document using Multi-Dimensional Scaling. The variable `pos` holds the `x` and `y`  coordinates.

In [29]:
mds = MDS(n_components=2, max_iter=3000,
      random_state=seed, dissimilarity="precomputed", n_jobs=1)

pos = mds.fit(dist).embedding_

Create a list of x values (coordinates) and a list of y values.

In [30]:
pos_x = [x for x, y in pos]
pos_y = [y for x, y in pos]

Create lists of the most prevalent topic, the probability of the most prevalent topic, and the text name for each document.

In [31]:
prevalent_topic = doc_topic_df.drop('text_name', axis=1).idxmax(axis=1)
probability = [doc_topic_df.ix[i][prevalent_topic[i]] for i in range(0, len(corpus))]
text_name = [name for name in doc_topic_df['text_name']]

Import bokeh and draw the visualization. The visualization provides various tools for further exploration:
- tooltips (provides topic, probability, text name and URL)
- box zoom
- wheel zoom
- pan
- reset
- link to document edition
- save the visualization

In [32]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool
from bokeh.plotting import figure, output_file, output_notebook, show
from collections import OrderedDict
output_notebook()

p = figure(
    plot_width=800, plot_height=800,
    tools="tap,pan,wheel_zoom,box_zoom,reset,save", 
    title="Topic Distribution MDS.\nSize of the circle represents prevalence of the topic")
p.add_tools(HoverTool(
        tooltips=[
            ("url", "http://oracc.org/" + "@id_text"),
            (("topic, probability"), ("@topic, @probability")),
            ("text name", "@text_name")
        ]
        ))

colormap = [('topic 1', "orange"), ('topic 2', "olive"), ('topic 3', "firebrick"), 
          ('topic 4', "gold"), ('topic 5', "red"), ('topic 6', "black"), ('topic 7', "green"), 
          ('topic 8', "blue"), ('topic 9', "purple"), ('topic 10', "darkslategray"), ('topic 11', "yellow"), 
          ('topic 12', "indigo"), ('topic 13', "blueviolet"), ('topic 14', "saddlebrown"), ('topic 15',"navy"), ('topic 16', 'orange'),
           ('topic 17', 'olive'), ('topic 18', 'firebrick'), ('topic 19', 'gold'), ('topic 20', 'red'), ('topic 21', 'black'),
            ('topic 22', 'green'), ('topic 23', 'blue'), ('topic 24', 'purple'), ('topic 25', 'darkslategray')]
colormap = OrderedDict(colormap)
colors = [colormap[n] for n in prevalent_topic]
color_list = [colormap[topic] for topic in colormap]
color_list += ["black"] * (len(colors) - len(color_list))
source = ColumnDataSource(data=dict(
        x=pos_x,
        y=pos_y,
        id_text=list(doc_topic_df.index.values),
        size = probability/max(probability)*25,
        probability = probability,
        topic = prevalent_topic,
        color = colors,
        orig_color = color_list,
        text_name = text_name
    ))

p.circle('x', 'y', color='color', fill_alpha=0.5, size='size', source=source)

url = "http://oracc.museum.upenn.edu/@id_text"
    

taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(p)

/Users/veldhuis/anaconda/lib/python3.6/site-packages/bokeh/core/json_encoder.py:52: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  NP_EPOCH = np.datetime64('1970-01-01T00:00:00Z')


Loading BokehJS ...

The following graph is essentially the same but uses two sliders to choose two topic. The idea is to color the dots (documents) that have high prevalence for those topics and leave the other dots gray. This version is particuarly useful when modeling with many topics. 

In [33]:
from bokeh.io import vform
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, CustomJS
from bokeh.models.widgets import Slider
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import widgetbox, column
from collections import OrderedDict
output_notebook()

p = figure(
    plot_width=800, plot_height=800,
    tools="tap,pan,wheel_zoom,box_zoom,reset,save", 
    title="Topic Distribution MDS\nSize of the circle represents prevalence of the topic")
p.add_tools(HoverTool(
        tooltips=[
            ("url", "http://oracc.org/" + "@id_text"),
            (("topic, probability"), ("@topic, @probability")),
            ("text name", "@text_name")
        ]
        ))


colormap = [('topic 1', "orange"), ('topic 2', "olive"), ('topic 3', "firebrick"), 
          ('topic 4', "gold"), ('topic 5', "red"), ('topic 6', "black"), ('topic 7', "green"), 
          ('topic 8', "blue"), ('topic 9', "purple"), ('topic 10', "darkslategray"), ('topic 11', "yellow"), 
          ('topic 12', "indigo"), ('topic 13', "blueviolet"), ('topic 14', "saddlebrown"), ('topic 15',"navy"), ('topic 16', 'orange'),
           ('topic 17', 'olive'), ('topic 18', 'firebrick'), ('topic 19', 'gold'), ('topic 20', 'red'), ('topic 21', 'black'),
            ('topic 22', 'green'), ('topic 23', 'blue'), ('topic 24', 'purple'), ('topic 25', 'darkslategray')]
colormap = OrderedDict(colormap)
colors = [colormap[n] for n in prevalent_topic]
color_list = [colormap[topic] for topic in colormap]
color_list += ["black"] * (len(colors) - len(color_list))

source = ColumnDataSource(data=dict(
        x=pos_x,
        y=pos_y,
        id_text=list(doc_topic_df.index.values),
        size = probability/max(probability)*25,
        probability = probability,
        topic = prevalent_topic,
        color = colors,
        orig_color = color_list,
        text_name = text_name
    ))

p.circle('x', 'y', color='color', fill_alpha=0.5, size='size', source=source)

callback = CustomJS(args=dict(source=source), code = """
    var data = source.get('data');
    var current_topic_1 = topic1.value;
    var current_topic_2 = topic2.value;
    x = data['x']
    y = data['y']
    col = data['color']
    topic = data['topic']

    orig_color = data['orig_color']
    for (i = 0; i < x.length; i++) {
        if (topic[i].substring(6) == current_topic_1) {
            col[i] = orig_color[current_topic_1 - 1]
        } else {
            col[i] = 'grey'
        }
        if (topic[i].substring(6) == current_topic_2) {
            col[i] = orig_color[current_topic_2 - 1]
        }
    }
    source.trigger('change');
""")

topic_slider = Slider(start=1, end = ntopics, value=1, step=1, title = 'Topic', callback=callback)
callback.args['topic1'] = topic_slider
topic_slider2 = Slider(start=1, end = ntopics, value=1, step=1, title = 'Topic', callback=callback)
callback.args['topic2'] = topic_slider2

url = "http://oracc.museum.upenn.edu/@id_text"

taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(column(topic_slider, topic_slider2, p))

Loading BokehJS ...

# Visualization 2

The visualization below does essentially the same thing: each document (dot) is colored according to the most prevalent topic and the size of the dot represents the probability of the most prevalent topic in that document. The position of the dot (embedding), in this case, is computed with t-SNE ([t-distributed Stochastic Neighbor Embedding](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)).
The coding in Bokeh is essentially the same and thus provides the same tools: links to the documents, tooltips, zooming, etc.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
X = doc_topic_df.drop('text_name', axis=1).as_matrix()
tsne = TSNE(n_components = 2, init = 'pca', random_state=0)
X_tsne = tsne.fit_transform(X)

In [ ]:
tsne_x = [x for x, y in X_tsne]
tsne_y = [y for x, y in X_tsne]

In [ ]:
from bokeh.io import vform
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, CustomJS
from bokeh.models.widgets import Slider
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import widgetbox, column
from collections import OrderedDict
output_notebook()

p = figure(
    plot_width=800, plot_height=800,
    tools="tap,pan,wheel_zoom,box_zoom,reset,save", 
    title="Topic Distribution tSNE.\nSize of the circle represents prevalence of the topic")
p.add_tools(HoverTool(
        tooltips=[
            ("url", "http://oracc.org/" + "@id_text"),
            (("topic, probability"), ("@topic, @probability")),
            ("text name", "@text_name")
        ]
        ))


colormap = [('topic 1', "orange"), ('topic 2', "olive"), ('topic 3', "firebrick"), 
          ('topic 4', "gold"), ('topic 5', "red"), ('topic 6', "black"), ('topic 7', "green"), 
          ('topic 8', "blue"), ('topic 9', "purple"), ('topic 10', "darkslategray"), ('topic 11', "yellow"), 
          ('topic 12', "indigo"), ('topic 13', "blueviolet"), ('topic 14', "saddlebrown"), ('topic 15',"navy"), ('topic 16', 'orange'),
           ('topic 17', 'olive'), ('topic 18', 'firebrick'), ('topic 19', 'gold'), ('topic 20', 'red'), ('topic 21', 'black'),
            ('topic 22', 'green'), ('topic 23', 'blue'), ('topic 24', 'purple'), ('topic 25', 'darkslategray')]
colormap = OrderedDict(colormap)
colors = [colormap[n] for n in prevalent_topic]
color_list = [colormap[topic] for topic in colormap]
color_list += ["black"] * (len(colors) - len(color_list))

source = ColumnDataSource(data=dict(
        x=tsne_x,
        y=tsne_y,
        id_text=list(doc_topic_df.index.values),
        size = probability/max(probability)*25,
        probability = probability,
        topic = prevalent_topic,
        color = colors,
        orig_color = color_list,
        text_name = text_name
    ))

p.circle('x', 'y', color='color', fill_alpha=0.5, size='size', source=source)

callback = CustomJS(args=dict(source=source), code = """
    var data = source.get('data');
    var current_topic_1 = topic1.value;
    var current_topic_2 = topic2.value;
    x = data['x']
    y = data['y']
    col = data['color']
    topic = data['topic']

    orig_color = data['orig_color']
    for (i = 0; i < x.length; i++) {
        if (topic[i].substring(6) == current_topic_1) {
            col[i] = orig_color[current_topic_1 - 1]
        } else {
            col[i] = 'grey'
        }
        if (topic[i].substring(6) == current_topic_2) {
            col[i] = orig_color[current_topic_2 - 1]
        }
    }
    source.trigger('change');
""")

topic_slider = Slider(start=1, end = ntopics, value=1, step=1, title = 'Topic', callback=callback)
callback.args['topic1'] = topic_slider
topic_slider2 = Slider(start=1, end = ntopics, value=1, step=1, title = 'Topic', callback=callback)
callback.args['topic2'] = topic_slider2

url = "http://oracc.museum.upenn.edu/@id_text"

taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(column(topic_slider, topic_slider2, p))